In [13]:
library(DESeq2)

In [14]:
df <- read.table("dea/sum.tsv", header=TRUE, sep="\t")
# Drop duplicates in id column
df <- df[!duplicated(df$id),]
rownames(df) <- df$id
df$id <- NULL
head(df)

,aging_30m_CYP19A1_18m_3,aging_18m_ESR1_no_3,aging_18m_CYP19A1_no_2,aging_18m_CYP19A1_no_3,aging_30m_ESR1_18m_1,aging_24m_CYP19A1_6m_2,aging_12m_ESR1_1w_1,antiHormonal_20m_ESR1_tamoxifen_2m_2,aging_18m_ESR1_1w_2,aging_18m_ESR1_1w_3,⋯,antiHormonal_20m_CYP19A1_tamoxifen_2m_1,aging_12m_ESR1_1w_2,aging_18m_ESR1_no_2,antiHormonal_20m_CYP19A1_letrozole_2m_1,antiHormonal_20m_CYP19A1_no_1,antiHormonal_20m_ESR1_tamoxifen_2m_3,aging_24m_CYP19A1_12m_2,antiHormonal_18m_CYP19A1_no_2,antiHormonal_18m_CYP19A1_no_1,antiHormonal_20m_CYP19A1_tamoxifen_2m_2
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
chr1:3729265-3729444,0,0,0,0,0,0,0,16,0,0,⋯,0,0,0,0,0,0,0,0,0,0
chr1:4212834-4298842,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,8,0,0,0,0,0
chr1:4212834-4337843,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
chr1:4234077-4301367,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
chr1:4267756-4298842,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,8,0,0,0,0,0,0
chr1:4267756-4301367,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,24


In [15]:
phenotype <- read.csv("dea/phenotype.csv", header=TRUE, row.names = 1)
head(phenotype)

,condition,age,transgene,induction,drug,replicate
,<chr>,<int>,<chr>,<int>,<chr>,<int>
aging_12m_ESR1_no_1,12_ESR1_0_no,12,ESR1,0,no,1
aging_12m_ESR1_no_2,12_ESR1_0_no,12,ESR1,0,no,2
aging_12m_ESR1_no_3,12_ESR1_0_no,12,ESR1,0,no,3
aging_12m_ESR1_1w_1,12_ESR1_1_no,12,ESR1,1,no,1
aging_12m_ESR1_1w_2,12_ESR1_1_no,12,ESR1,1,no,2
aging_12m_ESR1_1w_3,12_ESR1_1_no,12,ESR1,1,no,3


In [16]:
df <- df[, rownames(phenotype)]

In [18]:
dds <- DESeqDataSetFromMatrix(countData = df,
                              colData = phenotype,
                              design = ~ age + transgene + induction + drug)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  the design formula contains one or more numeric variables with integer values,
  specifying a model with increasing fold change for higher values.
  did you mean for this to be a factor? if so, first convert
  this variable to a factor using the factor() function

  the design formula contains one or more numeric variables that have mean or
  standard deviation larger than 5 (an arbitrary threshold to trigger this message).
  Including numeric variables with large mean can induce collinearity with the intercept.
  Users should center and scale numeric variables in the design to improve GLM convergence.

